# Summary of Results and Algorithm:


## Read input

In [56]:
# Imports
import pandas as pd
import numpy as np
# Read data into dataframe
df = pd.read_csv('./google-cluster-data-1.csv',sep=' ')

In [57]:
# Print the length of the df
print("length of df: ", len(df))

length of df:  3535029


Get CPU and MEM information

In [58]:
# Show amount of CPU for each task
df['NrmlTaskCores'].values

array([0.      , 0.      , 0.021875, ..., 0.      , 0.      , 0.      ])

In [59]:
# Save CPU amount, memory requirements, and task IDs
cpus = df['NrmlTaskCores'].values
mems = df['NrmlTaskMem'].values
task_ids = df['TaskID'].values

In [60]:
# Calculate and save execution times for two cases: mod 2 and mod 10
temp1 = df['TaskID'].values
ExecutionTime1 = []
for i in temp1:
    burstTime = (i%2+1)*300
    ExecutionTime1.append(burstTime)
    

In [61]:
# Calculate and print total # of subtasks for two cases: mod 2 and mod 10
totalSubtask1 = 0
for i in ExecutionTime1:
    totalSubtask1 += i/300
print("Total Subtasks for setting 1:", int(totalSubtask1))


Total Subtasks for setting 1: 5298274


In [62]:
# Save calculated execution times to columns in the dataframe
df['executionTime1'] = ExecutionTime1

In [63]:
# Show dataframe
df.head()

,Time,ParentID,TaskID,JobType,NrmlTaskCores,NrmlTaskMem,Unnamed: 6,executionTime1
0,90000,757745334,1488529826,0,0.000000,0.031130,NaN,300
1,90000,975992247,1488529821,0,0.000000,0.000000,NaN,600
2,90000,1468458091,1488529832,1,0.021875,0.002353,NaN,300
3,90000,1460281235,1488529840,0,0.000000,0.000000,NaN,300
4,90000,1164728954,1488529835,0,0.003125,0.001638,NaN,600


### Setting 1

In [16]:
# Calculates energy consumption for one vm
def getEnergyConsumption(VMCPUUsage, totalCPUCount, threshold):
    
    #here we want to calculate the cpu usage rate per individual VM so i look at the individual
    #cpu usage rate and divide it over 7/100 or 25/100
    cpuUsageRate = (totalCPUCount- VMCPUUsage)/(totalCPUCount)
    
    # Based on the equations provided
    static = 0
    if cpuUsageRate > 0:
        static = 5 
    else:
        static = 0
        
    dynamic = 0
    if cpuUsageRate < threshold:
        dynamic = cpuUsageRate*100
    else:
        dynamic = threshold * 100 + (((cpuUsageRate - threshold)**2) * 200)

    return static + dynamic

In [17]:
# Calculates cost for one vm by mapping possible time values to cost
def getCost(time, power):
    costChart = [0.5, 0.5, 0.6, 0.6, 0.6, 0.7, 0.7, 0.6, 0.6, 0.8, 0.8, 0.8, 0.8]
    mapping = [90000,91875,93750,95625,97500,99375,101250,103125,105000,106875,108750,110625]
    cost = 0
    for i in range(12):
        if time == mapping[i]:
            cost = costChart[i]
            break
        elif time < mapping[i]:
            cost = costChart[i-1]
        elif time > mapping[11]:
            cost = costChart[11]
    return cost*power
    

In [64]:
# Schedule all tasks in a way that minimizes the power consumption
def GreedyPower(input_data, nextQueue, threshhold, executionTime, cpuTotal, memTotal, final):
    length = len(input_data)
    nextQueue = []
    rejected =[]
    VMs = [[cpuTotal, memTotal] for i in range(100)]
    energyUsage = 0
    totalPower = 0
    totalCost = 0
    totalTurnaroundTime = 0
    
    # Iterates through the tasks
    for index, row in input_data.iterrows():
        cpu, mem = row['NrmlTaskCores'], row['NrmlTaskMem']
        task_id = row['TaskID']
        found_VM = False
        # Arbitrarely high value to compare the first energy value to 
        lowestPower = 234567898765
        lowestPowerIndex = -1
        VM_id = 0
        
        # Iterate through all of the vms
        for j in range(100):
            VM_id = j
            # Check whether this is even schedulable
            if VMs[VM_id][0] >= cpu and VMs[VM_id][1] >= mem:
                energy = getEnergyConsumption(VMs[VM_id][0], cpuTotal, threshhold)
                if energy < lowestPower:
                    lowestPower = energy
                    lowestPowerIndex = VM_id
                    
        # Is schedulable 
        if lowestPowerIndex != -1:
            cpu = row['NrmlTaskCores']
            mem = row['NrmlTaskMem']
            
            # Adjust the corresponding cpu and memory values for the VMs based on the allocated task
            VMs[lowestPowerIndex][0] = VMs[lowestPowerIndex][0] - cpu
            VMs[lowestPowerIndex][1] = VMs[lowestPowerIndex][1] - mem
            
            # Because we scheduled the task, update power, cost, and time
            totalPower += lowestPower
            totalCost += getCost(row['Time'], lowestPower)
            totalTurnaroundTime += row[executionTime]
            
        # If the task is rejected
        else:
            rejected.append(row['TaskID'])
            # Hard rejection of task by reducing the rest of the time for the subtasks to 0
            row[executionTime] = 0
                
        # Adjust remaining execution time for the overall task and append rest to nextQueue
        if row[executionTime] > 300:
            row[executionTime] -= 300
            nextQueue.append(row)
            
    if final == True:
        print(VMs)
    return rejected, nextQueue, totalPower, totalCost, totalTurnaroundTime

In [65]:
def GreedyCost(input_data, nextQueue, threshhold, executionTime,cpuTotal, memTotal, final):
    length = len(input_data)
    nextQueue = []
    VMs = [[cpuTotal, memTotal] for i in range(100)]
    cost = 0
    rejected =[]
    totalPower = 0
    totalCost = 0
    totalTurnaroundTime = 0
    
    for index, row in input_data.iterrows():
        cpu, mem = row['NrmlTaskCores'], row['NrmlTaskMem']
        task_id = row['TaskID']
        found_VM = False
        lowestCost = float('inf')
        lowestCostIndex = -1
        VM_id = 0
        
        #iterate through all of the vms
        for j in range(100):
            VM_id = j
            #check whether this is even schedulable
            if VMs[VM_id][0] >= cpu and VMs[VM_id][1] >= mem:
                energy = getEnergyConsumption(VMs[VM_id][0], cpuTotal, threshhold)
                cost = getCost(row['Time'], energy)
                if cost < lowestCost:
                    lowestCost = cost
                    lowestCostIndex = VM_id
                    
        #is schedulable
        if lowestCostIndex != -1:
            cpu = row['NrmlTaskCores']
            mem = row['NrmlTaskMem']
            
            VMs[lowestCostIndex][0] = VMs[lowestCostIndex][0] - cpu
            VMs[lowestCostIndex][1] = VMs[lowestCostIndex][1] - mem
            
            # Because we scheduled the task, update power, cost, and time
            totalPower += getEnergyConsumption(VMs[VM_id][0], cpuTotal, threshhold)
            totalCost += lowestCost
            totalTurnaroundTime += row[executionTime]
        else:
            rejected.append(row['TaskID'])
            # Hard rejection of task by reducing the rest of the time for the subtasks to 0
            row[executionTime] = 0
                
        if row[executionTime] > 300:
            row[executionTime] -= 300
            nextQueue.append(row)
            
    if final == True:
        print(VMs)
            
    return rejected, nextQueue, totalPower, totalCost, totalTurnaroundTime

In [66]:
# Schedule all tasks in a way that minimizes the turnover time
def GreedyTurnaroundTime(input_data, nextQueue, threshhold, executionTime,cpuTotal, memTotal,final):
    length = len(input_data)
    nextQueue = []
    VMs = [[cpuTotal, memTotal] for i in range(100)]
    energyUsage = 0
    rejected=[]
    totalPower = 0
    totalCost = 0
    totalTurnaroundTime = 0
    
    for index, row in input_data.iterrows():
        cpu, mem = row['NrmlTaskCores'], row['NrmlTaskMem']
        task_id = row['TaskID']
        found_VM = False
        index = -1
        VM_id = 0
        
        #iterate through all of the vms
        for j in range(100):
            VM_id = j
            #check whether this is even schedulable
            if VMs[VM_id][0] >= cpu and VMs[VM_id][1] >= mem:
                index = VM_id
                break
                    
        # Is schedulable 
        if index != -1:
            cpu = row['NrmlTaskCores']
            mem = row['NrmlTaskMem']
            
            VMs[index][0] = VMs[index][0] - cpu
            VMs[index][1] = VMs[index][1] - mem
            
            # Because we scheduled the task, update power, cost, and time
            power = getEnergyConsumption(VMs[index][0], cpuTotal, threshhold)
            totalPower += power
            totalCost += getCost(row['Time'], power)
            totalTurnaroundTime += row[executionTime]
        else:
            rejected.append(row['TaskID'])
            # Hard rejection of task by reducing the rest of the time for the subtasks to 0
            row[executionTime] = 0
                
        if row[executionTime] > 300:
            row[executionTime] -= 300
            nextQueue.append(row)
        
    if final == True:
        print(VMs)
            
    return rejected, nextQueue, totalPower, totalCost, totalTurnaroundTime

In [44]:
def RR(input_data, nextQueue, threshhold, executionTime,cpuTotal, memTotal, final):
    rejected = []
    
    length = len(input_data)
    nextQueue = []
    VMs = [[cpuTotal, memTotal] for i in range(100)]
    energyUsage = 0
    totalPower = 0
    totalCost = 0
    totalTurnaroundTime = 0
    
    # Iterate through all of the tasks
    
    for index, row in input_data.iterrows():
        cpu, mem = row['NrmlTaskCores'], row['NrmlTaskMem']
        task_id = row['TaskID']
        VM_id = 0
        for j in range(100):
            # find the available cpu and mem units based on VM_id
            cpu_vm = VMs[VM_id][0]
            mem_vm = VMs[VM_id][1] 
            # If a given VM has space, then remove the CPU and MEM units for the given task and go to the next task
            if cpu_vm >= cpu and mem_vm >= mem:
                # We can put task_id[i] into VM[VM_id]
                # update VM resource availability. 
                VMs[VM_id][0] = cpu_vm - cpu
                VMs[VM_id][1] = mem_vm - mem
                # Increase the VM_id so that the next VM can be checked
                VM_id = VM_id + 1
                # Reset VM_id back to 0 if it hits 100
                if VM_id == 100:
                    VM_id = 0
                # Because we scheduled the task, update power, cost, and time
                power = getEnergyConsumption(VMs[VM_id][0], cpuTotal, threshhold)
                totalPower += power
                totalCost += getCost(row['Time'], power)
                totalTurnaroundTime += row[executionTime]
                # Skip to the next task since this one has been allocated
                break
            # If on the last VM and the task still hasn't been alloted to a VM, then put the task id in rejected
            elif j == 99:
                rejected.append(row['TaskID'])
                # Hard rejection of task by reducing the rest of the time for the subtasks to 0
                row[executionTime] = 0
            # Increase the VM_id so that the next VM can be checked
            VM_id = VM_id + 1
            # Reset VM_id back to 0 if it hits 100
            if VM_id == 100:
                VM_id = 0
                
        if row[executionTime] > 300:
            row[executionTime] -= 300
            nextQueue.append(row)
            
    if final == True:
        print(VMs)


    # If none of the tasks were rejected, print 0
    if len(rejected) == 0:
        print('0')
        
    return rejected, nextQueue, totalPower, totalCost, totalTurnaroundTime

### Round Robin

In [45]:
# Run RR for CPU = 7, Mem = 11
VMs = [[7, 11] for i in range(100)]
setty = set(df['Time'].values)
setty = sorted(setty)
rejected = []
nextQueue = []
#print(len(setty))
totalPower = 0
totalCost = 0
totalTurnaroundTime = 0

for i in setty:
    taskQueue = df.loc[df['Time'] == i]
    taskQueue = taskQueue.append(nextQueue)
    final = False
    if i == 112500:
        final = True
    returnObj = RR(taskQueue, nextQueue, 0.5, 'executionTime1', 5, 10, final)
    rejected += returnObj[0]
    nextQueue = returnObj[1]
    totalPower = returnObj[2]
    #print(totalPower)
    totalCost = returnObj[3]
    totalTurnaroundTime = returnObj[4]

if len(rejected) > 0:
    np.save("taskReject_2_RR", rejected)
#np.save("VMs_1_iv", VMs)
print("Total Power: ", totalPower)
print("Total Cost: ", totalCost)
print("Total Turnaround Time: ", totalTurnaroundTime)

0
[[5.0, 2.801295029606409e-06], [5.0, 1.0201999895812415e-06], [5.0, 6.998599993518722e-06], [3.100000000000052, 5.054599988714405e-06], [1.7562500000000056, 1.55169999642088e-06], [3.206249999999998, 8.602700000109557e-06], [1.2781249999999995, 1.3180999716001069e-06], [0.0031249999999977563, 1.044169999859974e-05], [1.2616990785474513e-13, 1.8316000014320026e-05], [0.003124999999952348, 9.0680999992215e-05], [3.660873687527655e-14, 5.8542413679999905], [0.0031249999999479923, 8.672536419999993], [0.0031249999999996446, 9.33442469800001], [1.6374054889745082e-14, 9.18602754999999], [1.0656406312925526e-14, 9.33000007000001], [7.19303089313783e-15, 9.400869550000005], [0.003124999999996979, 9.492575689999995], [6.218983661376853e-16, 9.149561630000003], [1.2487406941819046e-14, 9.384226400000008], [7.059717394009013e-14, 1.9015207859999972], [1.7180701306074297e-14, 2.9400782480000003], [3.6637359812630166e-15, 5.589046468000005], [1.0816000872715392e-15, 6.6402285309999955], [0.00312

### Greedy Algorithms

In [68]:
# Run greedy power for CPU = 7, Mem = 11
VMs = [[7, 11] for i in range(100)]
setty = set(df['Time'].values)
setty = sorted(setty)
rejected = []
nextQueue = []
#print(len(setty))
totalPower = 0
totalCost = 0
totalTurnaroundTime = 0

# Run task queue based on optimization for power
for i in setty:
    taskQueue = df.loc[df['Time'] == i]
    taskQueue = taskQueue.append(nextQueue)
    #print(i)
    final = False
    if i == 112500:
        final = True
    returnObj = GreedyPower(taskQueue, nextQueue, 0.5, 'executionTime1', 5, 10, final)
    rejected += returnObj[0]
    nextQueue = returnObj[1]
    totalPower = returnObj[2]
    totalCost = returnObj[3]
    totalTurnaroundTime = returnObj[4]

# Save rejected values
if len(rejected) > 0:
    np.save("taskReject_2_1", rejected)
#np.save("VMs_1_i", VMs)
print("Total Power: ", totalPower)
print("Total Cost: ", totalCost)
print("Total Turnaround Time: ", totalTurnaroundTime)

[[5.0, 2.801295029606409e-06], [5.0, 1.0201999895812415e-06], [5.0, 6.998599993518722e-06], [2.7531250000000074, 1.2850313359999943], [2.765625000000008, 6.535397170000009], [2.765625000000008, 5.34378188200001], [2.765625000000007, 6.210028758000003], [2.7656250000000075, 6.561210094000007], [2.7625000000000073, 6.576865653999988], [2.7687500000000034, 6.871859618], [2.7687500000000047, 6.818682015999985], [2.765625000000007, 6.554561722000005], [2.768750000000006, 5.694814981999992], [2.7656250000000067, 6.671803149999999], [2.7656250000000075, 6.449514898000001], [2.759375000000004, 6.879104906000003], [2.7656250000000075, 6.558806590000008], [2.7593750000000044, 7.13629773599999], [2.7656250000000075, 6.171160337999996], [2.7406250000000063, 5.4223609308999725], [2.7562500000000063, 6.189715998000009], [2.7687500000000065, 6.566640771500006], [2.762500000000005, 7.033847376000001], [2.762500000000006, 6.958763695999997], [2.765625000000007, 7.118573442999999], [2.7687500000000065, 

In [69]:
# Run greedy cost for CPU = 7, Mem = 11
VMs = [[7, 11] for i in range(100)]
setty = set(df['Time'].values)
setty = sorted(setty)
rejected = []
nextQueue = []
totalPower = 0
totalCost = 0
totalTurnaroundTime = 0
counter = 0

for i in setty:
    taskQueue = df.loc[df['Time'] == i]
    taskQueue = taskQueue.append(nextQueue)
    final = False
    if i == 112500:
        final = True
    returnObj = GreedyCost(taskQueue, nextQueue, 0.5, 'executionTime1', 5, 10, final)
    rejected += returnObj[0]
    nextQueue = returnObj[1]
    totalPower = returnObj[2]
    totalCost = returnObj[3]
    totalTurnaroundTime = returnObj[4]

if len(rejected) > 0:
    np.save("taskReject_2_2", rejected)
#np.save("VMs_1_ii", VMs)
print("Total Power: ", totalPower)
print("Total Cost: ", totalCost)
print("Total Turnaround Time: ", totalTurnaroundTime)

[[5.0, 2.801295029606409e-06], [5.0, 1.0201999895812415e-06], [5.0, 6.998599993518722e-06], [2.7656250000000075, 1.3392531199999942], [2.765625000000008, 6.53478770000001], [2.7531250000000074, 5.264252968000009], [2.7687500000000065, 6.343391738000004], [2.7656250000000075, 6.561210094000007], [2.7625000000000073, 6.576865653999988], [2.7687500000000034, 6.8723085799999994], [2.765625000000005, 6.8515771159999845], [2.7656250000000075, 6.495114912000005], [2.768750000000006, 5.694814981999992], [2.7656250000000067, 6.694050617999999], [2.765625000000008, 6.472951508000002], [2.7687500000000043, 6.858776296000003], [2.7656250000000075, 6.558677630000007], [2.7625000000000046, 7.104241221999989], [2.765625000000007, 6.205024377999996], [2.7687500000000065, 5.504213732899977], [2.7687500000000065, 6.1309756700000095], [2.765625000000007, 6.576935825500005], [2.768750000000005, 6.892888604], [2.762500000000006, 6.958763695999997], [2.7656250000000058, 6.795202921999997], [2.75625000000000

In [70]:
# Run greedy turnaround time for CPU = 7, Mem = 11
VMs = [[7, 11] for i in range(100)]
setty = set(df['Time'].values)
setty = sorted(setty)
rejected = []
nextQueue = []
totalPower = 0
totalCost = 0
totalTurnaroundTime = 0

for i in setty:
    taskQueue = df.loc[df['Time'] == i]
    taskQueue = taskQueue.append(nextQueue)
    final = False
    if i == 112500:
        final = True
    returnObj = GreedyTurnaroundTime(taskQueue, nextQueue, 0.5, 'executionTime1', 5, 10, final)
    rejected += returnObj[0]
    nextQueue = returnObj[1]
    totalPower = returnObj[2]
    totalCost = returnObj[3]
    totalTurnaroundTime = returnObj[4]

if len(rejected) > 0:
    np.save("taskReject_2_3", rejected)
#np.save("VMs_1_iii", VMs)
print("Total Power: ", totalPower)
print("Total Cost: ", totalCost)
print("Total Turnaround Time: ", totalTurnaroundTime)

[[5.0, 2.801295029606409e-06], [5.0, 1.0201999895812415e-06], [5.0, 6.998599993518722e-06], [3.100000000000052, 5.054599988714405e-06], [1.7562500000000056, 1.55169999642088e-06], [3.206249999999998, 8.602700000109557e-06], [1.2781249999999995, 1.3180999716001069e-06], [0.0031249999999977563, 1.044169999859974e-05], [1.2616990785474513e-13, 1.8316000014320026e-05], [0.003124999999952348, 9.0680999992215e-05], [3.660873687527655e-14, 5.8542413679999905], [0.0031249999999479923, 8.672536419999993], [0.0031249999999996446, 9.33442469800001], [1.6374054889745082e-14, 9.18602754999999], [1.0656406312925526e-14, 9.33000007000001], [7.19303089313783e-15, 9.400869550000005], [0.003124999999996979, 9.492575689999995], [6.218983661376853e-16, 9.149561630000003], [1.2487406941819046e-14, 9.384226400000008], [7.059717394009013e-14, 1.9015207859999972], [1.7180701306074297e-14, 2.9400782480000003], [3.6637359812630166e-15, 5.589046468000005], [1.0816000872715392e-15, 6.6402285309999955], [0.0031249

In [71]:
print(len(rejected))

863259


In [ ]:
# RESULTS DISCUSSION
# For setting 1, the amount of power consumed is less when we use the algorithm that optimizes for power and the
# cost is incurred is less when we use the algorithm that optimizes for cost. The turnaround time is the same for all
# three algorithms because execution time is independent of cost and power. The same patterns apply to setting 2.


# Why optimizing cost and energy? What's the difference? 

# Optimizing for cost and energy is similar in the sense that cost in part depends on energy. However, cost also
# depends on time of day. So optimizing for energy purely depends on the VMs available, while optimizing for cost 
# depends on more parameters.


# When does it make a difference to optimize energy and cost? Provide a discussion.

# If the energy consumption is relatively stable over time, but there are high differences in price at certain times,
# then it makes a difference. This is because optimizing for cost would result in different results during times
# when the cost is higher than if just power was optimized for. In a real engineering scenario, it makes sense to 
# optimize depending on the greatest constraint.